In [1]:
import numpy as np
import pickle
import pandas as pd
import json
import os
import copy
import gc
import math
import scipy.sparse as sp

In [2]:
with open('data/plsss.pkl','rb') as f:
    mappings = pickle.load(f)
with open('data/user_handle.pkl','rb') as f:
    user_handle = pickle.load(f)
with open('data/user_id.pkl','rb') as f:
    user_id = pickle.load(f)
with open('data/problem_mappings.pkl','rb') as f:
    problem_id = pickle.load(f)
with open('data/problem_name.pkl','rb') as f:
    problem_name = pickle.load(f)

In [3]:
problem_AC = np.load('data/problem_AC.npy')
problem_attempted = np.load('data/problem_attempted.npy')
user_AC = np.load('data/user_AC.npy')
user_attempted = np.load('data/user_attempted.npy')
mean_user_rating = np.load('data/mean_user_rating.npy')

In [4]:
def get_problem_id(cid, idx):
    if str(cid)+idx in problem_id:
        return problem_id[str(cid)+idx]
    if cid>len(mappings) or len(idx)>1:
        return -1
    if mappings[cid] !=  0:
        #print(str(cid)+idx)
        idx = chr(int(ord(idx) - mappings[cid][1]))
        cid = cid + mappings[cid][0]
        #print('haha'+str(cid)+idx)
        if str(cid) + idx in problem_id:
            return problem_id[str(cid)+idx]
    return -1

In [5]:
# !pip install implicit

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
# !pip install annoy
# !pip install nmslib
import implicit
import annoy
import nmslib

In [30]:
# !conda install -c akode nmslib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /Users/aanyajindal/anaconda3

  added / updated specs: 
    - nmslib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.10.15         |           py36_0         138 KB
    ca-certificates-2018.03.07 |                0         124 KB
    numpy-1.14.2               |   py36ha9ae307_0         3.9 MB
    pybind11-2.2.4             |   py36h04f5b5a_0         114 KB
    openssl-1.0.2p             |       h1de35cc_0         3.4 MB
    nmslib-1.7.2               |           py36_0         1.2 MB  akode
    blas-1.0                   |              mkl           5 KB
    ------------------------------------------------------------
                

In [31]:
# conda install faiss-cpu -c pytorch

SyntaxError: invalid syntax (<ipython-input-31-39f21265b96f>, line 1)

In [7]:
sparse_user_item = sp.load_npz('data/confi.npz')
sparse_item_user = sp.load_npz('data/confi.npz')

In [8]:
sparse_item_user = sparse_item_user.transpose()
print(sparse_user_item)

  (0, 0)	1
  (0, 3)	2
  (0, 4)	3
  (0, 5)	6
  (0, 16)	1
  (0, 29)	1
  (0, 30)	1
  (0, 31)	1
  (0, 33)	1
  (0, 39)	1
  (0, 40)	1
  (0, 41)	2
  (0, 43)	20
  (0, 54)	1
  (0, 55)	1
  (0, 56)	1
  (0, 57)	7
  (0, 64)	1
  (0, 65)	4
  (0, 66)	2
  (0, 67)	2
  (0, 68)	2
  (0, 69)	1
  (0, 70)	1
  (0, 71)	1
  :	:
  (146975, 3034)	1
  (146975, 3035)	8
  (146975, 3040)	3
  (146975, 3045)	2
  (146975, 3049)	1
  (146975, 3057)	2
  (146975, 3062)	1
  (146975, 3069)	1
  (146975, 3070)	1
  (146975, 3118)	0
  (146975, 3119)	1
  (146975, 3120)	4
  (146975, 3124)	1
  (146975, 3125)	1
  (146975, 3138)	10
  (146975, 3139)	4
  (146975, 3145)	10
  (146975, 3146)	0
  (146975, 3190)	1
  (146975, 3830)	1
  (146975, 3832)	1
  (146975, 3833)	2
  (146975, 3866)	4
  (146975, 3885)	1
  (146975, 4225)	1


In [9]:
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
from implicit.datasets.lastfm import get_lastfm

/Users/aanyajindal/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
# maps command line model argument to class name
MODELS = {"als":  AlternatingLeastSquares,
          "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
          "faiss_als": FaissAlternatingLeastSquares,
          "tfidf": TFIDFRecommender,
          "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
          "bm25": BM25Recommender}

In [24]:
def get_model(model_name):
    print("getting model %s" % model_name)
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = { 'factors' : 30, 'regularization' : 0.1, 'iterations' : 20}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    elif model_name == "bpr":
        params = {'factors': 63}
    else:
        params = {}
    return model_class(**params)

In [36]:
TRAINED_MODELS = {} 
# TRAINED_MODELS["als"] = get_model("als");
for key in MODELS:
    TRAINED_MODELS[key] = get_model(key)

getting model als
getting model nmslib_als
getting model annoy_als
getting model faiss_als
getting model tfidf
getting model cosine
getting model bpr
getting model bm25


In [37]:
for key in MODELS:
    TRAINED_MODELS[key].fit(sparse_item_user)

100%|██████████| 20.0/20 [00:28<00:00,  1.33s/it]
100%|██████████| 20.0/20 [00:28<00:00,  1.40s/it]
100%|██████████| 20.0/20 [00:29<00:00,  1.41s/it]
100%|██████████| 20.0/20 [00:29<00:00,  1.39s/it]
100%|██████████| 4434/4434 [00:01<00:00, 2661.57it/s]


In [39]:
print("AlternatingLeastSquares")
rec = TRAINED_MODELS["als"].recommend(24593, sparse_user_item)
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

AlternatingLeastSquares
[(2047, 0.8525554), (3132, 0.8510778), (3660, 0.84038234), (3158, 0.829785), (3196, 0.8250598), (3195, 0.8066391), (3197, 0.7868161), (2457, 0.7807896), (3554, 0.7604027), (2447, 0.7504646)]
474D
734C
854B
742B
750B
750A
750C
580B
833A
577B


In [40]:
print("NMSLibAlternatingLeastSquares")
rec = TRAINED_MODELS["nmslib_als"].recommend(24593, sparse_user_item)
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

NMSLibAlternatingLeastSquares
[(3132, 0.8654004), (3660, 0.85219437), (3158, 0.8370004), (3196, 0.8302378), (3195, 0.8261272), (3197, 0.7920261), (3554, 0.77872986), (3071, 0.7385594), (3058, 0.73668236), (3562, 0.73470044)]
734C
854B
742B
750B
750A
750C
833A
723D
721B
835B


In [41]:
print("AnnoyAlternatingLeastSquares")
rec = TRAINED_MODELS["annoy_als"].recommend(24593, sparse_user_item)
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

AnnoyAlternatingLeastSquares
[(3132, 0.895273413071339), (3158, 0.8792915669818797), (2047, 0.866520672834261), (3196, 0.8351614125421153), (3195, 0.8287860693912628), (3660, 0.8154518457763651), (3554, 0.7947159648119714), (3197, 0.7919956345088313), (2457, 0.7762978018967764), (3562, 0.7487856968485501)]
734C
742B
474D
750B
750A
854B
833A
750C
580B
835B


In [42]:
print("FaissAlternatingLeastSquares")
rec = TRAINED_MODELS["annoy_als"].recommend(24593, sparse_user_item)
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

FaissAlternatingLeastSquares
[(3132, 0.8657158), (3196, 0.85909724), (3158, 0.85095733), (3660, 0.8430778), (2047, 0.8423544), (3197, 0.81303596), (3554, 0.7895785), (3225, 0.75530314), (2447, 0.75273013), (3071, 0.7195673)]
734C
750B
742B
854B
474D
750C
833A
757B
577B
723D


In [48]:
print("TFIDFRecommender")
rec = TRAINED_MODELS["tfidf"].recommend(24593, sparse_user_item.astype('double'))
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

TFIDFRecommender
[(2473, 12.130588167377903), (3057, 11.348712901024426), (2457, 10.272810301543416), (2441, 10.055864124403733), (3561, 9.875561198987693), (2634, 9.83492521260234), (2091, 9.795699199937069), (2647, 9.640222096949733), (3157, 9.504550740812363), (3554, 9.488534415691062)]
583B
721A
580B
576A
835A
618B
486C
621B
742A
833A


In [49]:
print("CosineRecommender")
rec = TRAINED_MODELS["cosine"].recommend(24593, sparse_user_item.astype('double'))
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

CosineRecommender
[(3195, 19.716005760134), (3196, 15.947029961941867), (3057, 15.342357193519097), (3320, 15.098618698913654), (3561, 14.78278733176253), (2473, 12.777909944862015), (3321, 11.524583498360043), (2441, 11.230204704176954), (3151, 10.896333245003017), (2091, 10.343204896258367)]
750A
750B
721A
776A
835A
583B
776B
576A
740B
486C


In [45]:
print("BayesianPersonalizedRanking")
rec = TRAINED_MODELS["bpr"].recommend(24593, sparse_user_item)
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

BayesianPersonalizedRanking
[(3109, 3.0798333), (3139, 2.666864), (3132, 2.5139813), (3158, 2.4891126), (3117, 2.4462817), (2047, 2.4455173), (3127, 2.4428802), (2477, 2.4390447), (3151, 2.4110217), (3162, 2.4006019)]
730J
735D
734C
742B
731F
474D
733D
584D
740B
743D


In [50]:
print("BM25Recommender")
rec = TRAINED_MODELS["bm25"].recommend(24593, sparse_user_item.astype('double'))
print(rec)
for i in range(len(rec)):
    print(problem_name[rec[i][0]])

BM25Recommender
[(3373, 2180982.172084487), (3554, 1939548.4559641036), (3158, 1862188.2311556651), (3533, 1710653.2252629686), (2664, 1654144.6392246545), (3271, 1651549.691309037), (3132, 1587830.7320572687), (2691, 1537678.833391359), (2457, 1535248.6028317816), (2616, 1432171.3908982624)]
789B
833A
742B
827A
625A
767B
734C
629D
580B
614B
